In [12]:
import time
import numpy as np
import scipy.stats
import sys
sys.path.append('/home/yilingh/SI-Interaction')

from selectinf.Simulation.spline_instance import generate_gaussian_instance_nonlinear_interaction_simple
from selectinf.group_lasso_query import group_lasso
from selectinf.reluctant_interaction import (SPAM, split_SPAM)
from selectinf.base import selected_targets_interaction
import seaborn as sns
import matplotlib.pyplot as plt
from simulation_helpers import (interaction_t_test_single,
                                interaction_t_tests_all,
                                interaction_t_tests_all_parallel)
from selectinf.Simulation.spline_instance import generate_gaussian_instance_from_bspline_interaction
from multiprocessing import Pool
from functools import partial

In [35]:
inst = generate_gaussian_instance_nonlinear_interaction_simple
p=30
intercept_flag = True
SNR = 1
(design, data_interaction, Y, Y_mean, data_combined,
         groups, active, active_inter_adj, active_inter_list) \
            = inst(n=500, p_nl=p, p_l=0, s_l=0,
                   nknots=6, degree=2, SNR=SNR,
                   rho=0.5, rho_noise=0.5, full_corr=False,
                   center=False, scale=False, random_signs=True,
                   intercept=intercept_flag, structure='weakhierarchy',
                   s_interaction=10, interaction_signal=2)

Equally spaced quantile knots used.


In [36]:
from simulation_helpers import MLE_inference_inter

In [37]:
conv, nonzero \
    = MLE_inference_inter(design, Y, Y_mean, groups,
                          n_features=p, interactions=data_interaction, 
                          intercept=False,
                          weight_frac=2, level=0.9,
                          proportion=0.5, mode="stronghierarchy",
                          parallel=False, continued=False, solve_only=True,
                          conv_cont=None, nonzero_cont=None, ncores=8
                        )

In [48]:
start = time.time()
coverages, lengths, selected_interactions \
    = MLE_inference_inter(design, Y, Y_mean, groups,
                          n_features=p, interactions=data_interaction, 
                          intercept=False,
                          weight_frac=2, level=0.9,
                          proportion=0.5, mode="weakhierarchy",
                          parallel=False, continued=True, solve_only=False,
                          conv_cont=conv, nonzero_cont=nonzero, ncores=8
                        )
end = time.time()
print(end - start)
print(np.mean(coverages))
print(np.mean(lengths))

34.75310015678406
0.9955555555555555
11.161891861489702


In [50]:
start = time.time()
coverages, lengths, selected_interactions1 \
    = MLE_inference_inter(design, Y, Y_mean, groups,
                          n_features=p, interactions=data_interaction, 
                          intercept=False,
                          weight_frac=2, level=0.9,
                          proportion=0.5, mode="weakhierarchy",
                          parallel=True, continued=True, solve_only=False,
                          conv_cont=conv, nonzero_cont=nonzero, ncores=2
                        )
end = time.time()
print(end - start)
print(np.mean(coverages))
print(np.mean(lengths))

/Users/yilingh/Desktop/PhD/SI_Codes/SI-Interaction/env3/lib/python3.9/site-packages/regreg/atoms/slope.py:16: UserWarning: unable to import isotonic regression from sklearn, using a pure python implementation
  warn('unable to import isotonic regression from sklearn, using a pure python implementation')
/Users/yilingh/Desktop/PhD/SI_Codes/SI-Interaction/env3/lib/python3.9/site-packages/regreg/atoms/slope.py:16: UserWarning: unable to import isotonic regression from sklearn, using a pure python implementation
  warn('unable to import isotonic regression from sklearn, using a pure python implementation')


22.189836740493774
0.9955555555555555
11.161891861489702


In [17]:
import time
import itertools

import pandas as pd
from simulation_helpers import (one_sim_mode, one_sim_mode_serial)
from multiprocessing import Pool

from functools import partial

In [18]:
def interaction_filter_vary_SNR(nsim=100, use_MLE=True, parallel=True,
                                ncores=8):
    """
    Compare to R randomized lasso
    """

    # Operating characteristics
    oper_char = {}
    oper_char["coverage rate"] = []
    oper_char["avg length"] = []
    oper_char["method"] = []
    oper_char["F1 score interaction"] = []
    oper_char["|G|"] = []
    oper_char["mode"] = []
    oper_char["SNR"] = []
    
    p = 30
    mode = 'weakhierarchy'
    intercept_flag = True
    
    """
    GOOD RESULT:
    1. 
    p = 30
    SNR = 1
    intercept_flag = True
    ### Partially correlated
    (design, data_interaction, Y, Y_mean, data_combined,
                 groups, active, active_inter_adj, active_inter_list) \
                    = inst(n=500, p_nl=p, p_l=0, s_l=0,
                           nknots=6, degree=2, SNR=SNR, rho=0.5, rho_noise=0.5,
                           center=False, scale=False, random_signs=True,
                           intercept=intercept_flag, structure='weakhierarchy', 
                           s_interaction=10, interaction_signal=2)
    weight_frac = 2
                           
    2. 
    p = 30
    SNR = 0.5
    intercept_flag = True
    ### Fully correlated
    (design, data_interaction, Y, Y_mean, data_combined,
                 groups, active, active_inter_adj, active_inter_list) \
                    = inst(n=500, p_nl=p, p_l=0, s_l=0,
                           nknots=6, degree=2, SNR=SNR, rho=0.5, rho_noise=0.5,
                           center=False, scale=False, random_signs=True,
                           intercept=intercept_flag, structure='weakhierarchy', 
                           s_interaction=10, interaction_signal=2)
    weight_frac = 2
    """
    if parallel:
        oper_char_list = []
        
    for SNR in [0.1, 0.5, 1, 2, 5]:  
        if parallel:
            with Pool(ncores) as pool:
                results = pool.map(partial(one_sim_mode, SNR=SNR, 
                                           intercept_flag=intercept_flag,
                                           p=p, use_MLE=use_MLE, mode=mode,
                                           weight_frac=1.5),
                                   list(range(nsim)))
            oper_char_list = oper_char_list + results
        else:
            for i in range(nsim):
                print(i, "th simulation for mode:", mode)
                
                one_sim_mode_serial(SNR=SNR, intercept_flag=intercept_flag,
                             p=p, oper_char=oper_char, use_MLE=use_MLE,
                             mode=mode)
    if parallel:
        oper_char_list = list(itertools.chain(*oper_char_list))
        oper_char = pd.DataFrame(oper_char_list)
        oper_char.columns = ["coverage rate","avg length","F1 score interaction",
                             "method", "|G|", "mode", "SNR"]
        return oper_char
    else:
        return pd.DataFrame(oper_char)

In [19]:
oper_char_SNR = interaction_filter_vary_SNR(80, use_MLE=False, parallel=False)

0 th simulation for mode: weakhierarchy
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
DS Selection Size: 35
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Equally spaced quantile knots used.
Eq

KeyboardInterrupt: 